# A binary classification Neural Network #

Purpose of this notebook :

- create a binary classification neural network from scratch
- practice oop programming
- practice docstring and commentaries

In [1]:
# Add useful librairies

import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
# First the full neural network

# The prupose is to predict from data
# let's create some data a XOR problematic

# Let's fix ou random seed
np.random.seed(42)

# A dataset of m example with 2 inputs shape(2 m)
X = np.random.randn(2, 100)
# Transorm inputs in O. or 1.
X = (X > 0.5) * 1.

# The labels (when x = (0 0) or (1 1) > False, x = (0 1) or (1 0) > True)
Y = (np.sum(X, axis=0, keepdims=True) == 1) * 1.

print('X : \n', X[:,:5])
print('Y : \n', Y[:,:5])

X : 
 [[0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0.]]
Y : 
 [[0. 0. 1. 1. 0.]]


In [16]:
X = np.array([[0,1,0,1],[1,1,0,0]])
Y = np.array([[1,0,0,1]])
print(X)
print(Y)

[[0 1 0 1]
 [1 1 0 0]]
[[1 0 0 1]]


In [44]:
# Initialisation
# List of hidden layers
hidden_layers = [3]

np.random.seed(42)

# List of layers for binary classification
layers = [X.shape[0]] + hidden_layers + [1]

# Initialisation of list Weights and bias shape (output_layer(-1) layer) (output_layer(-1) 1)
W = [np.random.randn(layers[k], layers[k-1]) for k in range(1, len(layers))]
b = [np.zeros((layers[k], 1)) for k in range(1, len(layers))]

print('layers : ', layers)
for j,i in enumerate(W):
    print('shape W',j,' :',i.shape)
for j, i in enumerate(b):
    print('shape b',j,' : ', i.shape)


layers :  [2, 3, 1]
shape W 0  : (3, 2)
shape W 1  : (1, 3)
shape b 0  :  (3, 1)
shape b 1  :  (1, 1)


In [18]:
# Coding the sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# testons la sigmoid
sigmoid(0)

0.5

In [45]:
# Forward

# Let's make a list of activation starting with the inputs matrix
activation = [X]

# Loop to pass through all neurons
for l in range(len(layers)-1):
#     print(activation[l],' * ',W[l],' + ',b)
    pre_activation = np.dot(W[l], activation[l]) + b[l]
#     print('preactivation :',pre_activation)
    activation.append(np.around(sigmoid(pre_activation),1))

# check the activations shape    
    print(activation[l].shape)
print(activation[-1].shape)

# Check activation values
print(activation[-1])
    

(2, 4)
(3, 4)
(1, 4)
[[0.8 0.8 0.7 0.8]]


In [46]:
# What about the loss and cost

prediction_inter = activation[-1]

# loss function
loss = -Y * np.log(prediction_inter) - (1 - Y) * np.log(1 - prediction_inter)
print(loss)

# number of examples
m = X.shape[1]
# Cost function
cost = np.sum(loss) / m

print(cost)

[[0.22314355 1.60943791 1.2039728  0.22314355]]
0.814924454847114


In [49]:
# Backward

# First the last layer
d_loss = -Y/activation[-1] + (1-Y)/(1-activation[-1])
d_activation = activation[-1] * (1 - activation[-1])
d_pre_activation = d_loss * d_activation

d_W = [np.dot(d_pre_activation, activation[-2].T)]
d_b = [np.sum(d_pre_activation, axis=1, keepdims=True) / m]

# Loop for the next layers
for l in reversed(range(1, len(layers)-1)):
    d_pre_activation = np.dot(W[l].T, d_pre_activation)
    d_pre_activation_sig = d_pre_activation * activation[l] * (1 - activation[l])
    
    d_W.insert(0, np.dot(d_pre_activation_sig, activation[l-1].T))
    d_b.insert(0, np.sum(d_pre_activation_sig, axis=1, keepdims=True) / m)

# We can check the shapes
for i in d_W:
    print('dw :', i.shape)
for i in d_b:
    print('db :', i.shape)
    

dw : (3, 2)
dw : (1, 3)
db : (3, 1)
db : (1, 1)


In [50]:
# updating the parameters

# Setting the learning rate
learning_rate = 0.5

# Loop for updating all weights and bias in the network
for i in range(len(layers)-1):
    W[i] -= learning_rate * d_W[i]
    b[i] -= learning_rate * d_b[i]
    
# Checking shapes
    print('new W', i, ' : ', W[i].shape)
    print('new b', i, ' : ', b[i].shape)
    

new W 0  :  (3, 2)
new b 0  :  (3, 1)
new W 1  :  (1, 3)
new b 1  :  (1, 1)
